In [111]:
#Implementing K-Means Algorithm and Expectation Maximization using GMM
#Group Member: Sandra Elizabeth Rajoo   USC ID: 5477806497

In [29]:
import pandas as pd
import numpy as np

In [30]:
df = pd.read_csv("clusters.txt", names = ['x', 'y'])

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x       150 non-null    float64
 1   y       150 non-null    float64
dtypes: float64(2)
memory usage: 2.5 KB


In [32]:
#K-Means

In [33]:
def kmeans(df, k):
    iterate = 1
    clusters = np.zeros(df.shape[0])
    centroids = df.sample(n = k).values
    while iterate:
        for idx in range(len(df)):
            point = (df.loc[idx, "x"], df.loc[idx, "y"])
            min_dist = float('inf')
            for idx_, centroid in enumerate(centroids):
                dist = np.sqrt((centroid[0] - float(point[0])) ** 2 + (centroid[1] - float(point[1])) ** 2)
                if min_dist > dist:
                    min_dist = dist
                    clusters[idx] = idx_
        new_centroids = pd.DataFrame(df).groupby(by = clusters).mean().values
        if np.count_nonzero(centroids - new_centroids) == 0:
            iterate = 0
        else:
            centroids = new_centroids
    return centroids, clusters

In [34]:
centroids, clusters = kmeans(df, 3)

In [35]:
centroids

array([[-1.0393701 , -1.23803927],
       [ 5.17290392,  4.13591368],
       [ 0.49711036,  1.26696375]])

In [36]:
#Expectation Maximization using GMM

In [37]:
from scipy.stats import multivariate_normal
class GMM:
    def __init__(self, k, max_iters = 100):
        self.k = k
        self.max_iters = max_iters
    
    def initialize(self, X):
        self.shape = X.shape
        self.n, self.m = X.shape
        self.phi = np.full(shape = self.k, fill_value = 1 / self.k)
        self.weights = np.random.uniform(low = 0, high = 1, size = self.shape)  
        
        rand_row = np.random.randint(low = 0, high = self.n, size = self.k)
        self.mu = [X[idx, :] for idx in rand_row]
        self.sigma = [np.cov(X.T) for _ in range(self.k)]
        self.amplitude = np.zeros(self.k)
        
    def e_step(self, X):
        self.weights = self.predict_probability(X)
        self.phi = self.weights.mean(axis=0)
    
    def m_step(self, X):
        for i in range(self.k):
            weight = self.weights[:, [i]]
            total_weight = weight.sum()
            self.mu[i] = (X * weight).sum(axis=0) / total_weight
            self.sigma[i] = np.cov(X.T, aweights = (weight / total_weight).flatten(), bias=True)
    
    def fit(self, X):
        self.initialize(X)
        for iteration in range(self.max_iters):
            self.e_step(X)
            self.m_step(X)
        
        return self.mu, self.sigma, self.amplitude
    
    def multivariate_normal(self, X, mu, sigma):
        denom = (((2 * np.pi) ** len(X)) * np.linalg.det(sigma)) ** (-1/2)
        exp_power = -np.dot(np.dot((X - mu), np.linalg.inv(sigma)), (X - mu)) / 2
        numerator = np.exp(exp_power)
        
        return numerator / denom
#         return (2*np.pi)**(-len(X)/2)*np.linalg.det(sigma)**(-1/2)*np.exp(-np.dot(np.dot((X-mu).T, np.linalg.inv(sigma)), (X-mu))/2)
    
    def predict_probability(self, X):
        likelihood = np.zeros((self.n, self.k))
        for i in range(self.k):
            distribution = multivariate_normal(mean = self.mu[i], cov = self.sigma[i])
            likelihood[:,i] = distribution.pdf(X)
        self.amplitude = np.sum(likelihood, axis = 0) / self.n
        numerator = likelihood * self.phi
        denominator = numerator.sum(axis=1)[:, np.newaxis]
        weights = numerator / denominator
        return weights
    
    def predict(self, X):
        weights = self.predict_proba(X)
        return np.argmax(weights, axis = 1)


In [38]:
gmm = GMM(k = 3)
mean, variance, amplitude = gmm.fit(df.values)

In [39]:
print("Mean: ")
print(mean)
print("Variance: ")
print(variance)
print("Amplitude: ")
print(amplitude)

Mean: 
[array([-1.38120342,  1.02034656]), array([-0.80902014, -1.15902515]), array([4.43734878, 3.34597868])]
Variance: 
[array([[1.49678059, 0.70856961],
       [0.70856961, 2.19351501]]), array([[1.01699997, 0.13185861],
       [0.13185861, 0.98843776]]), array([[3.4856666 , 2.31767455],
       [2.31767455, 5.18150075]])]
Amplitude: 
[0.01679727 0.03728992 0.01016129]
